In [17]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import f1_score
import numpy as np
from itertools import islice

In [2]:
data = []
isFirst = False
with open('../data/features.csv') as f:
    for line in islice(f,1,None):
        temp = line[0:-1].split(',')
        if isFirst is False:
            isFirst = True
            for i in range(len(temp)):
                data.append([])
                data[i].append(temp[i])
        else:
            for i in range(len(temp)):
                data[i].append(temp[i])

In [3]:
for i in range(1,len(data)):
    data[i] = [ float(x) for x in data[i] ]

In [10]:
logBB = []
with open('../data/logBB.txt') as f:
    for line in f:
        logBB.append(float(line[0:-2]))
    f.close()

In [11]:
y1 = [ 
    1 if i>=0.3
    else 0 
    for i in logBB
]
y2 = [
    1 if i>=-1
    else 0
    for i in logBB
]

In [69]:
train_x = [data[1], data[3]]
train_x = np.array(train_x)
train_x = train_x.transpose()
train_y = y1
lda = LinearDiscriminantAnalysis(solver="svd", store_covariance=True,tol=0.01)
y_pred = lda.fit(train_x, train_y).predict(train_x)
lda.score(train_x,train_y)
lda.means_

array([[ 73.55213198,   1.35761421],
       [ 40.09263636,   2.44978182]])

In [67]:
train_x = [data[1], data[3], data[7]]
train_x = np.array(train_x)
train_x = train_x.transpose()
train_y = y2
lda = LinearDiscriminantAnalysis(solver="svd", store_covariance=True,tol=0.01)
y_pred = lda.fit(train_x, train_y).predict(train_x)
lda.score(train_x,train_y)

0.88925081433224751

In [15]:
lda.scalings_

array([[-0.0217102 ],
       [ 0.23012746],
       [ 0.3672364 ]])

In [18]:
f1_score(train_y,y_pred)

0.62176165803108796

In [19]:
lda.fit(train_x, train_y)

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='svd', store_covariance=True, tol=0.01)